# Sentiment Analysis using - RNN model

In [1]:
import os
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam


In [2]:
# File path to JSON dataset
file_path = r'C:\Users\Administrator\Documents\GitHub\Datasets\yelp_academic_dataset_review.json'

In [3]:
# Load data in chunks
chunk_size = 5000
model_save_path = 'sentiment_analysis_rnn_model.h5'

In [4]:
# Initialize the model outside the loop
model = None  
epochs = 10

In [5]:
for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    # Accessing the data DataFrame
    texts = chunk['text'].tolist()
    labels = chunk['stars'].tolist()

    # Tokenize the text
    tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    # Padding sequences
    max_len = 100 
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Convert labels to numerical format
    numeric_labels = np.array(labels)

    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(padded_sequences, numeric_labels, test_size=0.2, random_state=42)

    # Build or load the RNN model
    if model is None or not os.path.exists(model_save_path):
        embedding_dim = 50
        model = Sequential()
        model.add(Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len))
        model.add(LSTM(100))
        model.add(Dense(1, activation='linear'))  # Used linear activation for regression
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')  # Use mean squared error for regression
    else:
        # Load the previously trained model
        model = load_model(model_save_path)

    # Train the model
    model.fit(x_train, y_train, epochs=epochs, batch_size=128, validation_split=0.1)


Epoch 1/10
29/29 [==============================] - 6s 99ms/step - loss: 7.4102 - val_loss: 2.1837
Epoch 2/10
29/29 [==============================] - 2s 78ms/step - loss: 1.8583 - val_loss: 1.9495
Epoch 3/10
29/29 [==============================] - 2s 78ms/step - loss: 1.8092 - val_loss: 1.9221
Epoch 4/10
29/29 [==============================] - 2s 76ms/step - loss: 1.7743 - val_loss: 1.9038
Epoch 5/10
29/29 [==============================] - 2s 78ms/step - loss: 1.7472 - val_loss: 1.8866
Epoch 6/10
29/29 [==============================] - 2s 83ms/step - loss: 1.7006 - val_loss: 1.8751
Epoch 7/10
29/29 [==============================] - 2s 78ms/step - loss: 1.6588 - val_loss: 1.8637
Epoch 8/10
29/29 [==============================] - 2s 85ms/step - loss: 1.5758 - val_loss: 1.8426
Epoch 9/10
29/29 [==============================] - 2s 78ms/step - loss: 1.4670 - val_loss: 1.7832
Epoch 10/10
29/29 [==============================] - 2s 77ms/step - loss: 1.3222 - val_loss: 1.7420
Epoch 1/1

29/29 [==============================] - 3s 95ms/step - loss: 1.8446 - val_loss: 1.8495
Epoch 4/10
29/29 [==============================] - 3s 105ms/step - loss: 1.8261 - val_loss: 1.8312
Epoch 5/10
29/29 [==============================] - 3s 111ms/step - loss: 1.7923 - val_loss: 1.7999
Epoch 6/10
29/29 [==============================] - 3s 114ms/step - loss: 1.7409 - val_loss: 1.7745
Epoch 7/10
29/29 [==============================] - 3s 100ms/step - loss: 1.6737 - val_loss: 1.7699
Epoch 8/10
29/29 [==============================] - 3s 110ms/step - loss: 1.5871 - val_loss: 1.7367
Epoch 9/10
29/29 [==============================] - 3s 97ms/step - loss: 1.3560 - val_loss: 1.9591
Epoch 10/10
29/29 [==============================] - 3s 94ms/step - loss: 1.4702 - val_loss: 1.7828
Epoch 1/10
29/29 [==============================] - 7s 134ms/step - loss: 6.7435 - val_loss: 1.7874
Epoch 2/10
29/29 [==============================] - 3s 100ms/step - loss: 1.8118 - val_loss: 1.6905
Epoch 3/10
29

KeyboardInterrupt: 

In [ ]:
# Evaluate the final model on the test set
mse = model.evaluate(x_test, y_test)
print(f'Test Mean Squared Error: {mse}')

In [ ]:
# Save the entire model to an HDF5 file
model.save('final_rnn_model')